In [ ]:
import pathlib as pl
import pywatershed as pws

In [ ]:
output_dir = pl.Path("./07_cascading_flow/output")
domain_dir = pl.Path("../test_data/sagehen_5yr/")

In [ ]:
control_file = domain_dir / "sagehen_no_gw_cascades.control"
control = pws.Control.load_prms(control_file)
control.options["parameter_file"] = domain_dir / control.options["parameter_file"][2:]
control.options["input_dir"] = domain_dir / "output"
control.options["budget_type"] = None
control.options['netcdf_output_dir'] = output_dir
control.options['netcdf_output_var_names'] = [
    *pws.PRMSRunoffCascadesNoDprst.get_variables(),
    *pws.PRMSSoilzoneCascadesNoDprst.get_variables(),
    *pws.PRMSGroundwaterNoDprst.get_variables(),
]

In [ ]:
parameters = pws.parameters.PrmsParameters.load(control.options["parameter_file"])

In [ ]:
# add the cascading parameters via pre-processing
parameters = pws.utils.preprocess_cascades.preprocess_cascade_params(control, parameters)

In [ ]:
test_data_dir = pl.Path("../test_data")
domain_dir = test_data_dir / "drb_2yr"
dis_hru = None

model_dict = {
    "control": control,
    "dis_hru": dis_hru,
    "model_order": [
        "runoff",
        "soilzone",
        "groundwater",
    ],
    "runoff": {
        "class": pws.PRMSRunoffCascadesNoDprst,
        "parameters": parameters,
        "dis": "dis_hru",
    },
    "soilzone": {
        "class": pws.PRMSSoilzoneCascadesNoDprst,
        "parameters": parameters,
        "dis": "dis_hru",
    },
    "groundwater": {
        "class": pws.PRMSGroundwaterNoDprst,
        "parameters": parameters,
        "dis": "dis_hru",
    },
}

In [ ]:
model = pws.Model(model_dict)

In [ ]:
# model.run(netcdf_dir=output_dir, finalize=True)

In [ ]:
import xarray as xr
horton_casc = xr.open_dataarray(output_dir / "hru_horton_cascflow.nc")
sz_casc = xr.open_dataarray(output_dir / "hru_sz_cascadeflow.nc")

In [ ]:
proc_plot = pws.analysis.process_plot.ProcessPlot(
    "/Users/jmccreight/usgs/data/sagehen_pws_domain/GIS/", 
    hru_shp_file_name = "HRUs.shp",
    seg_shp_file_name = None,
)
proc_plot.plot_hru_var(
    var_name="hru_horton_cascflow",
    process=pws.PRMSRunoffCascadesNoDprst,
    data=horton_casc.mean(dim="time"),
    # data=horton_casc,  # make this a move in the underlying code
    data_units=horton_casc.attrs["units"],
    nhm_id=horton_casc["nhm_id"],
    clim = (0.0, 1.0e-2)
)

In [ ]:
proc_plot.plot_hru_var(
    var_name=var,
    process=pws.PRMSRunoffCascadesNoDprst,
    # data=horton_casc.mean(dim="time"),
    data=sz_casc[30, :],
    data_units=horton_casc.attrs["units"],
    nhm_id=horton_casc["nhm_id"],
)

In [ ]:
proc_plot.plot_hru_var(
    var_name=var,
    process=pws.PRMSRunoffCascadesNoDprst,
    # data=horton_casc.mean(dim="time"),
    data=sz_casc[135, :],
    data_units=horton_casc.attrs["units"],
    nhm_id=horton_casc["nhm_id"],
)